In [141]:
import pandas as pd
from utils import *
from mappings import *

In [142]:
df = read_dbf('../data/raw/exped.DBF')

In [143]:
df.head()

,expid,peakid,year,season,host,route1,route2,route3,route4,nation,...,accidents,achievment,agency,comrte,stdrte,primrte,primmem,primref,primid,chksum
0,ANN260101,ANN2,1960,1,1,NW Ridge-W Ridge,None,None,None,UK,...,None,None,None,None,None,False,False,None,None,2442047
1,ANN269301,ANN2,1969,3,1,NW Ridge-W Ridge,None,None,None,Yugoslavia,...,Draslar frostbitten hands and feet,None,None,None,None,False,False,None,None,2445501
2,ANN273101,ANN2,1973,1,1,W Ridge-N Face,None,None,None,Japan,...,None,None,None,None,None,False,False,None,None,2446797
3,ANN278301,ANN2,1978,3,1,N Face-W Ridge,None,None,None,UK,...,None,None,None,None,None,False,False,None,None,2448822
4,ANN279301,ANN2,1979,3,1,N Face-W Ridge,NW Ridge of A-IV,None,None,UK,...,None,None,None,None,None,False,False,None,None,2449204


### Primary key

In [144]:
df.expid = df.expid.str.cat(df.year.astype(str), sep='_')

In [145]:
df.head()

,expid,peakid,year,season,host,route1,route2,route3,route4,nation,...,accidents,achievment,agency,comrte,stdrte,primrte,primmem,primref,primid,chksum
0,ANN260101_1960,ANN2,1960,1,1,NW Ridge-W Ridge,None,None,None,UK,...,None,None,None,None,None,False,False,None,None,2442047
1,ANN269301_1969,ANN2,1969,3,1,NW Ridge-W Ridge,None,None,None,Yugoslavia,...,Draslar frostbitten hands and feet,None,None,None,None,False,False,None,None,2445501
2,ANN273101_1973,ANN2,1973,1,1,W Ridge-N Face,None,None,None,Japan,...,None,None,None,None,None,False,False,None,None,2446797
3,ANN278301_1978,ANN2,1978,3,1,N Face-W Ridge,None,None,None,UK,...,None,None,None,None,None,False,False,None,None,2448822
4,ANN279301_1979,ANN2,1979,3,1,N Face-W Ridge,NW Ridge of A-IV,None,None,UK,...,None,None,None,None,None,False,False,None,None,2449204


### Expedition Countries
#### Host

In [146]:
# replace documentation id mapping with custom mapping
df.host = apply_map(df.host, host_map)
df = update_country_list(df, 'host')

#### Nation

In [147]:
# explode slash-separated values into scalar names
df_nations = df[['expid', 'nation']]
df_nations.nation = df_nations.nation.str.split('/')
df_nations = df_nations.explode('nation', ignore_index=True)

In [148]:
# swap names with country ids
df_nations = update_country_list(df_nations, 'nation')

#### Countries

In [149]:
df.countries.head()

0    India, Nepal
1            None
2            None
3            None
4            None
Name: countries, dtype: object

In [150]:
# explode comma-separated country names into scalar values
df_countries = df[['expid', 'countries']].rename({'countries': 'country'}, axis=1)

df_countries.country = df_countries.country.str.split(',')
df_countries = df_countries.explode('country', ignore_index=True)

df_countries.country = df_countries.country.str.split('/')
df_countries = df_countries.explode('country', ignore_index=True)

In [151]:
df_countries.country.unique()

array(['India', ' Nepal', None, 'New Zealand', 'Nepal', 'USA', 'Austria',
       ' Italy', ' Switzerland', 'UK', ' France', ' W Germany',
       'Switzerland', 'France', 'Netherlands', 'Germany', ' Spain', ' UK',
       ' USA', 'Australia', 'Ireland', ' Netherlands', 'Czechoslovakia',
       'Canada', 'Belgium', ' Poland', 'Poland', 'China', 'Italy',
       'W Germany', 'Spain', ' Canada', ' Denmark', 'Brazil', 'Iceland',
       'Liechtenstein', 'Andorra', ' Ireland', 'Luxembourg', ' Mexico',
       'Argentina', ' India', 'Mexico', ' Belgium', ' New Zealand',
       ' Sweden', 'Sweden', 'Japan', 'Norway', 'S Korea', ' Germany',
       ' Portugal', ' Czechoslovakia', 'Bulgaria', 'Ukraine', 'Finland',
       ' Luxembourg', ' USSR', 'USSR', ' Norway', ' Hong Kong',
       'Indonesia', 'Ecuador', ' Yugoslavia', ' Kenya', 'Denmark',
       'Chile', 'S Africa', ' Israel', 'Kazakhstan', 'Croatia',
       ' Bulgaria', 'N Korea', ' Latvia', ' Peru', 'Portugal', 'Georgia',
       'Belarus', ' Sl

In [152]:
# swap names with country ids
df_countries = update_country_list(df_countries, 'country')

In [153]:
df.drop('countries', axis=1, inplace=True)

### Routes

In [154]:
route_1_df = df.loc[
	df.route1.notna() | df.success1.notna() | df.ascent1.notna(),
	['expid', 'route1', 'ascent1', 'success1']
]
route_1_df.rename({'route1': 'route', 'ascent1': 'ascent', 'success1': 'success'}, axis=1, inplace=True)
route_1_df['number'] = 1

In [155]:
route_2_df = df.loc[
	df.route2.notna() | df.success2.notna() | df.ascent2.notna(),
	['expid', 'route2', 'ascent2', 'success2']
]
route_2_df.rename({'route2': 'route', 'ascent2': 'ascent', 'success2': 'success'}, axis=1, inplace=True)
route_2_df['number'] = 2

In [156]:
route_3_df = df.loc[
	df.route3.notna() | df.success3.notna() | df.ascent3.notna(),
	['expid', 'route3', 'ascent3', 'success3']
]
route_3_df.rename({'route3': 'route', 'ascent3': 'ascent', 'success3': 'success'}, axis=1, inplace=True)
route_3_df['number'] = 3

In [157]:
route_4_df = df.loc[
	df.route4.notna() | df.success4.notna() | df.ascent4.notna(),
	['expid', 'route4', 'ascent4', 'success4']
]
route_4_df.rename({'route4': 'route', 'ascent4': 'ascent', 'success4': 'success'}, axis=1, inplace=True)
route_4_df['number'] = 4

In [158]:
df_routes = pd.concat([route_1_df, route_2_df, route_3_df, route_4_df], ignore_index=True)

In [159]:
df_routes.head()

,expid,route,ascent,success,number
0,ANN260101_1960,NW Ridge-W Ridge,1st,True,1
1,ANN269301_1969,NW Ridge-W Ridge,2nd,True,1
2,ANN273101_1973,W Ridge-N Face,3rd,True,1
3,ANN278301_1978,N Face-W Ridge,None,False,1
4,ANN279301_1979,N Face-W Ridge,None,False,1


In [ ]:
# TODO: drop route columnbs from main df